<a href="https://colab.research.google.com/github/saurabh1mohite/TDVR/blob/main/GraphDiffToTransformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [2]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [3]:
# # Turn a Unicode string to plain ASCII, thanks to
# # https://stackoverflow.com/a/518232/2809427
# def unicodeToAscii(s):
#     return ''.join(
#         c for c in unicodedata.normalize('NFD', s)
#         if unicodedata.category(c) != 'Mn'
#     )

# # Lowercase, trim, and remove non-letter characters

In [4]:
def readLangs(lang1, lang2, lang3, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('/content/drive/MyDrive/NLP/Trance/CSV/dataset.txt', encoding='utf-8').read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[s for s in l.split(',')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang_1 = Lang(lang2)
        input_lang_2 = Lang(lang1)
        output_lang = Lang(lang3)
    else:
        input_lang_1 = Lang(lang1)
        input_lang_2 = Lang(lang2)
        output_lang = Lang(lang3)
    return input_lang_1, input_lang_2, output_lang, pairs

In [5]:
def prepareData(lang1, lang2, lang3, reverse=False):
    input_lang_1, input_lang_2, output_lang, pairs = readLangs(lang1, lang2, lang3, reverse)
    print(lang1)
    print("Read %s sentence pairs" % len(pairs))
    # pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    # ctr = 0
    for pair in pairs:
        # print('pair - ', pair)
        # ctr += 1
        # if ctr > 10:
        #     break
        input_lang_1.addSentence(pair[0])
        input_lang_1.addSentence(pair[1])
        # input_lang_1.addSentence(pair[2])
        
        input_lang_2.addSentence(pair[1])
        # input_lang_2.addSentence(pair[2])
        input_lang_2.addSentence(pair[0])

        output_lang.addSentence(pair[2])
        # output_lang.addSentence(pair[1])
        # output_lang.addSentence(pair[0])
        
        # for word in pair[0].split(' '):
        #     input_lang_1.addWord(word)
        # for word in pair[1].split(' '):
        #     input_lang_2.addWord(word)
        # for word in pair[2].split(' '):
        #     output_lang.addWord(word)

    print("Counted words:")
    print(input_lang_1.name, input_lang_1.n_words)
    print(input_lang_2.name, input_lang_2.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang_1, input_lang_2, output_lang, pairs


input_lang_1, input_lang_2, output_lang, pairs = prepareData('init', 'final', 'transformation', True)
print(random.choice(pairs))

Reading lines...
init
Read 31800 sentence pairs
Trimmed to 31800 sentence pairs
Counting words...
Counted words:
final 256194
init 256194
transformation 45
['4 3 5 4 2 4 2 4 4 5 4 1 2 1 4 1 3 4 4 2 4 4 4 4 1 c8200661feb7927861f5a7f5f6b51e1d 1e736e30f50dfe42581bdf1c20181487 44502c72f4d0fa22832333a7efa43831 dc179b89ab1d21903def8758075b43ed 0b053511e0180e19cb9f472b1726fb0b fa375423bafd7b48b3e177c660d880e9 0b053511e0180e19cb9f472b1726fb0b dc179b89ab1d21903def8758075b43ed 4f955ef8fbb714f52e749d537ea2ff53 44502c72f4d0fa22832333a7efa43831 385099f1ea1f119e20af2765e0bd6ed2 34b8a5e9772cd73daba26c68a209127f 0b053511e0180e19cb9f472b1726fb0b 34b8a5e9772cd73daba26c68a209127f 38a8cf581986627f062b87819e90938f 34b8a5e9772cd73daba26c68a209127f 1e736e30f50dfe42581bdf1c20181487 dc179b89ab1d21903def8758075b43ed 4f955ef8fbb714f52e749d537ea2ff53 0b053511e0180e19cb9f472b1726fb0b 090a3276bf357cc2ba00aec70bef1c8f ee744c0b6a7285536ec2e8f3364d5d60 89af402e42075dcd347caa4f937e31b7 38a8cf581986627f062b87819e90938f 

In [6]:
class EncoderRNNL1(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNNL1, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [7]:
class EncoderRNNL2(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNNL2, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [8]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [9]:
MAX_LENGTH = 128
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [10]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor_1 = tensorFromSentence(input_lang_1, pair[0])
    input_tensor_2 = tensorFromSentence(input_lang_2, pair[1])
    target_tensor = tensorFromSentence(output_lang, pair[2])
    return (input_tensor_1, input_tensor_2, target_tensor)

In [11]:
teacher_forcing_ratio = 0.5


def train(input_tensor_1, input_tensor_2, target_tensor, encoder_1, encoder_2, decoder, encoder_optimizer_1, encoder_optimizer_2, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden_1 = encoder_1.initHidden()
    encoder_hidden_2 = encoder_2.initHidden()

    encoder_optimizer_1.zero_grad()
    encoder_optimizer_2 .zero_grad()
    decoder_optimizer.zero_grad()

    input_length_1 = input_tensor_1.size(0)
    input_length_2 = input_tensor_2.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs_1 = torch.zeros(max_length, encoder_1.hidden_size, device=device)
    encoder_outputs_2 = torch.zeros(max_length, encoder_2.hidden_size, device=device)

    loss = 0
################
    input_tensor_1 = input_tensor_1.to(device)
    input_tensor_2 = input_tensor_2.to(device)
##############
    for ei in range(input_length_1):
        encoder_output_1, encoder_hidden_1 = encoder_1(input_tensor_1[ei], encoder_hidden_1)
        encoder_outputs_1[ei] = encoder_output_1[0, 0]
    for ei in range(input_length_2):
        encoder_output_2, encoder_hidden_2 = encoder_2(input_tensor_2[ei], encoder_hidden_2)
        encoder_outputs_2[ei] = encoder_output_2[0, 0]
    encoder_outputs = encoder_outputs_1 - encoder_outputs_2

    decoder_input = torch.tensor([[SOS_token]], device=device)

# marker
    decoder_hidden = encoder_hidden_1 - encoder_hidden_2

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer_1.step()
    encoder_optimizer_2.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [12]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [13]:
import time
from tqdm.notebook import tqdm
def trainIters(encoder_1, encoder_2, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer_1 = optim.SGD(encoder_1.parameters(), lr=learning_rate)
    encoder_optimizer_2 = optim.SGD(encoder_2.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()
    for iter in tqdm(range(1, n_iters + 1)):
        training_pair = training_pairs[iter - 1]
        input_tensor_1 = training_pair[0]
        input_tensor_2 = training_pair[1]
        target_tensor = training_pair[2]

        loss = train(input_tensor_1, input_tensor_2, target_tensor, encoder_1, encoder_2,
                     decoder, encoder_optimizer_1, encoder_optimizer_2, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss
        
        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))
            torch.save(encoder_1.state_dict(), '/content/drive/MyDrive/NLP/Trance/models/encoder_weights_1')
            torch.save(encoder_2.state_dict(), '/content/drive/MyDrive/NLP/Trance/models/encoder_weights_2')
            torch.save(decoder.state_dict(), '/content/drive/MyDrive/NLP/Trance/models/decoder_weights')
            torch.save(encoder_optimizer_1.state_dict(), '/content/drive/MyDrive/NLP/Trance/models/encoder_optimizer_1')
            torch.save(encoder_optimizer_2.state_dict(), '/content/drive/MyDrive/NLP/Trance/models/encoder_optimizer_2')
            torch.save(decoder_optimizer.state_dict(), '/content/drive/MyDrive/NLP/Trance/models/decoder_optimizer')
        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0


    showPlot(plot_losses)

In [14]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [ ]:
hidden_size = 256
encoder_1 = EncoderRNNL1(input_lang_1.n_words, hidden_size).to(device)
encoder_2 = EncoderRNNL2(input_lang_2.n_words, hidden_size).to(device)
attn_decoder = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder_1, encoder_2, attn_decoder, 75000, print_every=100)

  0%|          | 0/75000 [00:00<?, ?it/s]

2m 38s (- 1975m 6s) (100 0%) 2.6793
5m 16s (- 1971m 18s) (200 0%) 1.9432
7m 40s (- 1912m 21s) (300 0%) 1.7593
10m 4s (- 1879m 59s) (400 0%) 1.5596
12m 28s (- 1859m 42s) (500 0%) 1.5777
14m 54s (- 1847m 57s) (600 0%) 1.7863
17m 17s (- 1835m 47s) (700 0%) 1.4989
19m 41s (- 1827m 4s) (800 1%) 1.6151
22m 6s (- 1820m 34s) (900 1%) 1.6562
24m 31s (- 1814m 16s) (1000 1%) 1.6552
26m 54s (- 1807m 24s) (1100 1%) 1.6046
29m 19s (- 1803m 2s) (1200 1%) 1.5851
31m 42s (- 1797m 58s) (1300 1%) 1.6392
34m 7s (- 1794m 14s) (1400 1%) 1.6576
36m 31s (- 1789m 55s) (1500 2%) 1.5843


In [ ]:
# modl arch, performance, analyze few examples, common patterns that are failing, overcoming existing baselines

In [ ]:
def evaluate(encoder_1, encoder_2, decoder, sentence_1, sentence_2, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor_1 = tensorFromSentence(input_lang_1, sentence_1)
        input_tensor_2 = tensorFromSentence(input_lang_2, sentence_2)
        input_length_1 = input_tensor_1.size()[0]
        input_length_2 = input_tensor_2.size()[0]
        encoder_hidden_1 = encoder_1.initHidden()
        encoder_hidden_2 = encoder_2.initHidden()

        encoder_outputs_1 = torch.zeros(max_length, encoder_1.hidden_size, device=device)
        encoder_outputs_2 = torch.zeros(max_length, encoder_2.hidden_size, device=device)
        

        for ei in range(input_length_1):
            encoder_output_1, encoder_hidden_1 = encoder_1(input_tensor_1[ei],
                                                     encoder_hidden_1)
            encoder_outputs_1[ei] += encoder_output_1[0, 0]
            
        for ei in range(input_length_2):
            encoder_output_2, encoder_hidden_2 = encoder_2(input_tensor_2[ei],
                                                     encoder_hidden_2)
            encoder_outputs_2[ei] += encoder_output_2[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        encoder_outputs = encoder_outputs_1 - encoder_outputs_2
        
        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]